# メールモジュールについて
Nginxはwebサービスの提供のみならず、メールサービスに関するプロキシ機能も提供しています。
ここでは
- 基本的なプロキシサービス
- 認証サービス
- memcachedとの連携
- ログファイルの解析
- OSによる制約
の5つに関してみていきましょう。

## 基本的なプロキシサービス
Nginxのメールプロキシモジュールは当初FastMail用に開発されました。

FastMailの仕組みは
- 利用者に単一のIMAPサーバを提供する必要がある。
- メールアカウント本体は、多数の上位メールサーバのいずれか１台に存在している。
- プロキシソフトウェアはUNIXのforkモデルを採用していた。 => 各コネクションに対して、プロセスを新規にforkしていた。
...
=> IMAPのコネクションは非常に長時間存在するため、これらのプロセスも長時間存在することになります。
よって、各コネクションが存在する限り、これらのプロセスを管理し続ける必要があり、プロキシサーバは非常に重くなります。

Nginxのイベントベースのプロセスモデルはこうしたサービスに適合しており、メールプロキシとして、メールアカウント本体が存在する多数のメールボックスサーバーに対するトラフィックの処理が可能です。
メールプロキシは多数のユーザーに対し、多数のメールボックスサーバーが存在している環境において、ユーザーからの通信を単一のサーバで対応する機能を提供します。
例として、
- Atmail
- Zimbra
がこのモデルを利用しています。

   
  | 基本的なプロキシサービスのモデル |     |                    |    |                          |    |                |
 | -------------                     | --- | ------------------ | -- | ------------------------ | -- | -------------- |
 | インターネット                    |     |                    |    |                          |    |                |
 |                                   | ->  | Nginx              |    |                          |    |                |
 |                                   |     |                    | -> | HTTP認証サービス         |    |                |
 |                                   |     |                    | -> | POP3メールボックスサーバ |    |                |
 |                                   |     |                    | -> | IMAPメールボックスサーバ |    |                |
 |                                   |     |                    | -> | SMTP中継ホスト           |    |                |
 |                                   |     |                    |    |                          | -> | インターネット |